[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/deepmipt/RDLS_NLP_2022/blob/main/lectures/04.Creating_Dialog_Systems/seminar/notebooks/07.pre_response_processing.ipynb)

In [ ]:
# !pip install df_engine df_runner df_db_connector df_telegram_connector

<img src="https://raw.githubusercontent.com/kudep/RDLS_images/bba12c082827415f7d3e1ab09dfb94d68415be93/dfe_examples/script_structire.svg" >
<img src="https://raw.githubusercontent.com/kudep/RDLS_images/bba12c082827415f7d3e1ab09dfb94d68415be93/dfe_examples/node_sructire.svg" >

In [ ]:
from df_engine.core import Actor, Context
import df_engine.labels as lbl


def print_function_response(argument: str):
    def response(ctx: Context, actor: Actor, *args, **kwargs):
        return argument

    return response


def create_transitions():
    return {
        ("left", "step_2"): "left",
        ("right", "step_2"): "right",
        lbl.to_start(): "start",  # to start node
        lbl.forward(): "forward",  # to next node in dict
        lbl.backward(): "back",  # to previous node in dict
        lbl.previous(): "previous",  # to previously visited node
        lbl.repeat(): "repeat",  # to the same node
        lbl.to_fallback(): cnd.true(),  # to fallback node
    }


def add_prefix(prefix):
    def add_prefix_processing(ctx: Context, actor: Actor, *args, **kwargs) -> Context:
        processed_node = ctx.current_node
        if not callable(processed_node.response):
            processed_node.response = f"{prefix}: {processed_node.response}"
        elif callable(processed_node.response):
            processed_node.response = f"{prefix}: {processed_node.response(ctx, actor, *args, **kwargs)}"
        ctx.overwrite_current_node_in_processing(processed_node)
        return ctx

    return add_prefix_processing

In [ ]:
# imports
from df_engine.core.keywords import GLOBAL, LOCAL, RESPONSE
from df_engine.core.keywords import TRANSITIONS, PRE_RESPONSE_PROCESSING
import df_engine.conditions as cnd

def response_handler(ctx: Context, actor: Actor, *args, **kwargs) -> bool:
    return f"answer {len(ctx.requests)}"


# a dialog script

# a dialog script
script = {
    "root": {
        "start": {RESPONSE: "", TRANSITIONS: {("flow", "step_0"): cnd.true()}},
        "fallback": {RESPONSE: "the end"},
    },
    GLOBAL: {
        PRE_RESPONSE_PROCESSING: {
            "proc_name_1": add_prefix("l1_global"),
            "proc_name_2": add_prefix("l2_global"),
        }
    },
    "flow": {
        LOCAL: {
            PRE_RESPONSE_PROCESSING: {
                "proc_name_2": add_prefix("l2_local"),
                "proc_name_3": add_prefix("l3_local"),
            }
        },
        "step_0": {
            RESPONSE: print_function_response("first"),
            TRANSITIONS: {lbl.forward(): cnd.true()},
        },
        "step_1": {
            PRE_RESPONSE_PROCESSING: {"proc_name_1": add_prefix("l1_step_1")},
            RESPONSE: print_function_response("second"),
            TRANSITIONS: {lbl.forward(): cnd.true()},
        },
        "step_2": {
            PRE_RESPONSE_PROCESSING: {"proc_name_2": add_prefix("l2_step_2")},
            RESPONSE: print_function_response("third"),
            TRANSITIONS: {lbl.forward(): cnd.true()},
        },
        "step_3": {
            PRE_RESPONSE_PROCESSING: {"proc_name_3": add_prefix("l3_step_3")},
            RESPONSE: print_function_response("fourth"),
            TRANSITIONS: {lbl.forward(): cnd.true()},
        },
        "step_4": {
            PRE_RESPONSE_PROCESSING: {"proc_name_4": add_prefix("l4_step_4")},
            RESPONSE: print_function_response("fifth"),
            TRANSITIONS: {"step_0": cnd.true()},
        },
    },
}


In [ ]:
from df_runner import ScriptRunner

runner = ScriptRunner(
    script,
    start_label=("root", "start"),
    fallback_label=("root", "fallback"),
)

In [ ]:
runner.start()